In [1]:
# all codes come are changed based on LDR code.
import argparse
import torch.optim as optim
import os
import yaml
from mcrgan.default import _C as config
from mcrgan.default import update_config
import torch
from mcrgan.trainer import MCRTrainer
from mcrgan.datasets import get_dataloader
from mcrgan.models import get_models
from mcrgan.models import get_BNN_models
from mcrgan.models import get_noise
from mcrgan.models import DiscriminatorMNIST
from mcrgan.models import weights_init_mnist_model_noise

import numpy as np
import torch
import argparse
from mcrgan.datasets import get_dataloader
from mcrgan.models import get_models
from mcrgan.default import _C as config
from mcrgan.default import update_config
from utils.utils import sort_dataset, compute_accuracy, extract_features
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from tqdm import tqdm
import os
from mcrgan.loss import MCRGANloss
import random
random.seed(0)

In [2]:
# data_preprocessing and parameters

device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
DATASET = 'mnist'
ROOT    = 'pth/to/the/dataset'
BATCH_SIZE = 2048
WORKERS = 0
DATA_SAMPLE = 50000
LABEL = 0

dataloader, dataset = get_dataloader(
        data_name=DATASET,
        root=ROOT,
        batch_size=BATCH_SIZE,
        num_workers=WORKERS)


In [3]:
import collections

def get_netG(netD,netV,net_noise,radius=1):
    models = [netD, netV,net_noise]
    worker_state_dict = [x.state_dict() for x in models]
    weight_keys = list(worker_state_dict[0].keys())
    fed_state_dict = collections.OrderedDict()
    for key in weight_keys:
        fed_state_dict[key] = worker_state_dict[0][key] + radius*worker_state_dict[2][key]*torch.log(1+torch.exp(worker_state_dict[1][key]))
    return fed_state_dict

# new netG already optimized by adam and time learning rate, so we only using netG to update netV
def update_netV(netV,netG,net_noise,fed_state_dict,radius=1):
    models =[netV,netG,net_noise]
    worker_state_dict = [x.state_dict() for x in models]
    weight_keys = list(worker_state_dict[0].keys())
    update_dict = collections.OrderedDict()
    for key in weight_keys:
        update_dict[key] = worker_state_dict[0][key] - (worker_state_dict[1][key]-fed_state_dict[key])*(worker_state_dict[2][key]*radius)/torch.log(1+torch.exp(-worker_state_dict[0][key]))
    return update_dict
        

In [4]:
#models
# train hyperparameters, batch norm not used var
#store_Var = []
import time
from torch_mimicry.training import scheduler, logger, metric_log

from test_acc import nearsub
import numpy as np
import torch
import argparse
from mcrgan.datasets import get_dataloader
from mcrgan.models import get_models
from mcrgan.default import _C as config
from mcrgan.default import update_config
from utils.utils import sort_dataset, compute_accuracy, extract_features2
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from tqdm import tqdm
import os

start_time = time.time()
#global_step = 0
num_steps = 150
n_dis = 1
n_dis2 = 1
numclasses =10
lr1 = 1e-3
lr2 = 1e-3
log_data = metric_log.MetricLog()
data_time = time.time()
iter_dataloader = iter(dataloader)
data, label = next(iter_dataloader)
data_time = time.time() - data_time
real_cpu = data.to(device)
real_label = label.clone().detach()

netD, netG = get_models(DATASET, device)

optD = optim.Adam(netD.parameters(), lr1, betas=(0.5, 0.999))
optG = optim.Adam(netG.parameters(), lr2, betas=(0.5, 0.999))

Acc_results_D = []
Acc_results_G = []




transform = transforms.Compose(
            [transforms.Resize(32),
             transforms.ToTensor(),
             transforms.Normalize(0.5, 0.5)])

trainset = datasets.MNIST(root=ROOT, train=True, download=True, transform=transform)
testset = datasets.MNIST(root=ROOT, train=False, download=True, transform=transform)

testloader = torch.utils.data.DataLoader(
        testset, batch_size=DATA_SAMPLE, shuffle=False, num_workers=WORKERS)

trainloader = torch.utils.data.DataLoader(
        trainset, batch_size=DATA_SAMPLE, shuffle=False, num_workers=WORKERS)
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
n_comp = 10


In [5]:
netD, netG

(DataParallel(
   (module): DiscriminatorMNIST(
     (main): Sequential(
       (0): Conv2d(1, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
       (1): LeakyReLU(negative_slope=0.2, inplace=True)
       (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
       (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (4): LeakyReLU(negative_slope=0.2, inplace=True)
       (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
       (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (7): LeakyReLU(negative_slope=0.2, inplace=True)
       (8): Conv2d(256, 128, kernel_size=(4, 4), stride=(1, 1), bias=False)
       (9): Flatten(start_dim=1, end_dim=-1)
     )
   )
 ),
 DataParallel(
   (module): GeneratorMNIST(
     (main): Sequential(
       (0): ConvTranspose2d(128, 256, kernel_size=(4, 4), stride=(1, 1), bias=False)
       (1)

In [6]:
real_cpu.shape

torch.Size([2048, 1, 32, 32])

In [7]:
netD(real_cpu).shape

torch.Size([2048, 128])

In [8]:
netG(netD(real_cpu)).shape

torch.Size([2048, 1, 32, 32])

In [9]:
netD(netG(netD(real_cpu))).shape

torch.Size([2048, 128])

In [10]:
netD(netG(netD(real_cpu)))

tensor([[ 0.0794, -0.1036,  0.0830,  ...,  0.0996,  0.0138, -0.0269],
        [ 0.0459,  0.0592,  0.1056,  ..., -0.0254,  0.0643, -0.0080],
        [ 0.0109, -0.1458, -0.1350,  ...,  0.0360,  0.1522,  0.1777],
        ...,
        [-0.0779,  0.0329,  0.0471,  ...,  0.0908, -0.0093,  0.0229],
        [ 0.1021,  0.0030, -0.0347,  ...,  0.0111,  0.0455,  0.0840],
        [-0.0282, -0.0318,  0.0101,  ...,  0.1470,  0.2507, -0.0418]],
       device='cuda:0', grad_fn=<DivBackward0>)

In [11]:
# fixed radius case, due to this requires more steps to go to about -200 then set 1500 epoches

n_dis = 1
n_dis2 = 1
mode = 1
gam3 = 1
mcr_gan_loss = MCRGANloss(gam1=1., gam2=1., gam3=gam3, eps=0.5, numclasses=numclasses, mode=mode, rho=None)

num_steps = 500
lr1 = 1e-3
lr2 = 1e-3
optD = optim.Adam(netD.parameters(), lr1, betas=(0.5, 0.999))
optG = optim.Adam(netG.parameters(), lr2, betas=(0.5, 0.999))
meanV_sum = []
radius_list = []
radius = 0

for epoch in range(num_steps):
    for i in range(n_dis):
        netD.zero_grad()
        optD.zero_grad()
        Z = netD(real_cpu)
        fake_img = netG(netD(real_cpu)).detach()
        Z_bar = netD(fake_img)
        errD, errD_EC = mcr_gan_loss(Z, Z_bar, real_label, 1, n_dis)
        print(errD, errD_EC)
        errD.backward()
        optD.step()
        
    for j in range(n_dis2):
        netG.zero_grad()
        optG.zero_grad()
        
        Z = netD(real_cpu)
        fake_img = netG(netD(real_cpu)).detach()
        Z_bar = netD(fake_img)
        
        errG, errD_EC = mcr_gan_loss(Z, Z_bar, real_label, 1, n_dis2)
        print(errG, errD_EC)
        errG = - errG
        errG.backward()
        optG.step()

    
    

tensor(-71.9471, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-9.4087, device='cuda:0', grad_fn=<NegBackward0>), tensor(-8.1438, device='cuda:0', grad_fn=<NegBackward0>), tensor(-54.3946, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-40.2581, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-4.6051, device='cuda:0', grad_fn=<NegBackward0>), tensor(-3.7450, device='cuda:0', grad_fn=<NegBackward0>), tensor(-31.9081, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-40.2581, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-4.6051, device='cuda:0', grad_fn=<NegBackward0>), tensor(-3.7450, device='cuda:0', grad_fn=<NegBackward0>), tensor(-31.9080, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-63.8624, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-6.7923, device='cuda:0', grad_fn=<NegBackward0>), tensor(-7.3672, device='cuda:0', grad_fn=<NegBackward0>), tensor(-49.7028, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-63.8623, device='cuda:0', grad_fn=<NegBackward0>) [tenso

tensor(-132.3202, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-14.6477, device='cuda:0', grad_fn=<NegBackward0>), tensor(-11.1295, device='cuda:0', grad_fn=<NegBackward0>), tensor(-106.5430, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-132.3202, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-14.6477, device='cuda:0', grad_fn=<NegBackward0>), tensor(-11.1295, device='cuda:0', grad_fn=<NegBackward0>), tensor(-106.5431, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-136.6490, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-15.0782, device='cuda:0', grad_fn=<NegBackward0>), tensor(-11.3744, device='cuda:0', grad_fn=<NegBackward0>), tensor(-110.1964, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-136.6490, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-15.0782, device='cuda:0', grad_fn=<NegBackward0>), tensor(-11.3744, device='cuda:0', grad_fn=<NegBackward0>), tensor(-110.1964, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-135.7536, device='cuda:0', grad_fn=<NegB

tensor(-148.4213, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-16.6957, device='cuda:0', grad_fn=<NegBackward0>), tensor(-11.3817, device='cuda:0', grad_fn=<NegBackward0>), tensor(-120.3438, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-141.7616, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-15.9849, device='cuda:0', grad_fn=<NegBackward0>), tensor(-10.8751, device='cuda:0', grad_fn=<NegBackward0>), tensor(-114.9016, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-141.7617, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-15.9849, device='cuda:0', grad_fn=<NegBackward0>), tensor(-10.8751, device='cuda:0', grad_fn=<NegBackward0>), tensor(-114.9017, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-139.7033, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-15.8896, device='cuda:0', grad_fn=<NegBackward0>), tensor(-10.5416, device='cuda:0', grad_fn=<NegBackward0>), tensor(-113.2721, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-139.7033, device='cuda:0', grad_fn=<NegB

tensor(-161.1876, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-18.1558, device='cuda:0', grad_fn=<NegBackward0>), tensor(-12.0072, device='cuda:0', grad_fn=<NegBackward0>), tensor(-131.0245, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-161.1876, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-18.1558, device='cuda:0', grad_fn=<NegBackward0>), tensor(-12.0072, device='cuda:0', grad_fn=<NegBackward0>), tensor(-131.0245, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-146.5448, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-16.8054, device='cuda:0', grad_fn=<NegBackward0>), tensor(-10.7843, device='cuda:0', grad_fn=<NegBackward0>), tensor(-118.9551, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-146.5448, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-16.8054, device='cuda:0', grad_fn=<NegBackward0>), tensor(-10.7843, device='cuda:0', grad_fn=<NegBackward0>), tensor(-118.9551, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-138.3462, device='cuda:0', grad_fn=<NegB

tensor(-173.2720, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-19.1196, device='cuda:0', grad_fn=<NegBackward0>), tensor(-12.9134, device='cuda:0', grad_fn=<NegBackward0>), tensor(-141.2390, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-173.1666, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-19.1120, device='cuda:0', grad_fn=<NegBackward0>), tensor(-12.8878, device='cuda:0', grad_fn=<NegBackward0>), tensor(-141.1668, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-173.1666, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-19.1120, device='cuda:0', grad_fn=<NegBackward0>), tensor(-12.8878, device='cuda:0', grad_fn=<NegBackward0>), tensor(-141.1668, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-174.0244, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-19.2299, device='cuda:0', grad_fn=<NegBackward0>), tensor(-13.0321, device='cuda:0', grad_fn=<NegBackward0>), tensor(-141.7623, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-174.0244, device='cuda:0', grad_fn=<NegB

...

tensor(-203.7508, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-30.1361, device='cuda:0', grad_fn=<NegBackward0>), tensor(-22.1703, device='cuda:0', grad_fn=<NegBackward0>), tensor(-151.4445, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-204.2156, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-30.1537, device='cuda:0', grad_fn=<NegBackward0>), tensor(-22.3737, device='cuda:0', grad_fn=<NegBackward0>), tensor(-151.6882, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-204.2156, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-30.1537, device='cuda:0', grad_fn=<NegBackward0>), tensor(-22.3737, device='cuda:0', grad_fn=<NegBackward0>), tensor(-151.6882, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-203.1602, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-30.1295, device='cuda:0', grad_fn=<NegBackward0>), tensor(-22.1609, device='cuda:0', grad_fn=<NegBackward0>), tensor(-150.8699, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-203.1602, device='cuda:0', grad_fn=<NegB

tensor(-198.0287, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-29.6434, device='cuda:0', grad_fn=<NegBackward0>), tensor(-21.2510, device='cuda:0', grad_fn=<NegBackward0>), tensor(-147.1343, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-198.0287, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-29.6434, device='cuda:0', grad_fn=<NegBackward0>), tensor(-21.2510, device='cuda:0', grad_fn=<NegBackward0>), tensor(-147.1343, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-200.4157, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-29.8978, device='cuda:0', grad_fn=<NegBackward0>), tensor(-21.5548, device='cuda:0', grad_fn=<NegBackward0>), tensor(-148.9631, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-200.4158, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-29.8978, device='cuda:0', grad_fn=<NegBackward0>), tensor(-21.5548, device='cuda:0', grad_fn=<NegBackward0>), tensor(-148.9632, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-202.2192, device='cuda:0', grad_fn=<NegB

tensor(-203.4451, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-30.5554, device='cuda:0', grad_fn=<NegBackward0>), tensor(-22.7421, device='cuda:0', grad_fn=<NegBackward0>), tensor(-150.1476, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-202.7245, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-30.4472, device='cuda:0', grad_fn=<NegBackward0>), tensor(-22.5089, device='cuda:0', grad_fn=<NegBackward0>), tensor(-149.7684, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-202.7245, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-30.4472, device='cuda:0', grad_fn=<NegBackward0>), tensor(-22.5090, device='cuda:0', grad_fn=<NegBackward0>), tensor(-149.7684, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-200.4989, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-30.2333, device='cuda:0', grad_fn=<NegBackward0>), tensor(-22.2461, device='cuda:0', grad_fn=<NegBackward0>), tensor(-148.0196, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-200.4989, device='cuda:0', grad_fn=<NegB

tensor(-213.7188, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-31.4999, device='cuda:0', grad_fn=<NegBackward0>), tensor(-24.3748, device='cuda:0', grad_fn=<NegBackward0>), tensor(-157.8441, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-213.7188, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-31.4999, device='cuda:0', grad_fn=<NegBackward0>), tensor(-24.3748, device='cuda:0', grad_fn=<NegBackward0>), tensor(-157.8440, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-213.4261, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-31.5499, device='cuda:0', grad_fn=<NegBackward0>), tensor(-24.3531, device='cuda:0', grad_fn=<NegBackward0>), tensor(-157.5230, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-213.4261, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-31.5499, device='cuda:0', grad_fn=<NegBackward0>), tensor(-24.3531, device='cuda:0', grad_fn=<NegBackward0>), tensor(-157.5230, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-213.0725, device='cuda:0', grad_fn=<NegB

tensor(-196.2852, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-29.8715, device='cuda:0', grad_fn=<NegBackward0>), tensor(-21.3590, device='cuda:0', grad_fn=<NegBackward0>), tensor(-145.0548, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-198.7199, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-30.1640, device='cuda:0', grad_fn=<NegBackward0>), tensor(-21.7203, device='cuda:0', grad_fn=<NegBackward0>), tensor(-146.8356, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-198.7200, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-30.1640, device='cuda:0', grad_fn=<NegBackward0>), tensor(-21.7203, device='cuda:0', grad_fn=<NegBackward0>), tensor(-146.8356, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-200.8468, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-30.3258, device='cuda:0', grad_fn=<NegBackward0>), tensor(-22.2038, device='cuda:0', grad_fn=<NegBackward0>), tensor(-148.3172, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-200.8468, device='cuda:0', grad_fn=<NegB

In [12]:
#acc at first for NetD
NetD_acc = 0
train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features(testloader, netD, netG)

test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features(testloader, netD, netG)


print("----------------------------")
print("Train_z, Test_z")
nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

#acc at first for NetD
from sklearn.neighbors import KNeighborsClassifier 
knn= KNeighborsClassifier(n_neighbors=10) 

knn.fit(train_Z,train_labels) 
knn.score(test_Z,test_labels) 
 
y_predict = knn.predict(test_Z) 

count =0
for k in range(10000):
    if y_predict[k] == test_labels[k]:
        count +=1
        
NetD_acc = count/10000
print('NetD, Acc at first', NetD_acc)
Acc_results_D.append(count/10000)

extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:00<00:00,  1.08it/s]


----------------------------
Train_z, Test_z
PCA: 0.9331
SVD: 0.9238
NetD, Acc at first 0.9657
